In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# 1.Data Preperation

In [2]:
df=pd.read_excel("E:\Machine Learning\HomeWork3\Application on Churn Rate\default of credit card clients.xls")
df.to_csv (r'E:\Machine Learning\HomeWork3\Application on Churn Rate\default of credit card clients.csv', index = None, header=True)
df=df.drop('ID', axis=1)


In [3]:
df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [4]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [5]:
df.columns

Index(['limit_bal', 'sex', 'education', 'marriage', 'age', 'pay_0', 'pay_2',
       'pay_3', 'pay_4', 'pay_5', 'pay_6', 'bill_amt1', 'bill_amt2',
       'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6', 'pay_amt1',
       'pay_amt2', 'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6',
       'default_payment_next_month'],
      dtype='object')

#### Attribute Information:

X1: Amount of the given credit (NT dollar): it includes both the individual consumer credit and his/her family (supplementary) credit.<br>
X2: Gender (1 = male; 2 = female).<br>
X3: Education (1 = graduate school; 2 = university; 3 = high school; 4 = others).<br>
X4: Marital status (1 = married; 2 = single; 3 = others).<br>
X5: Age (year).<br>
X6 - X11: History of past payment. We tracked the past monthly payment records (from April to September, 2005) as follows: X6 = the repayment status in September, 2005; X7 = the repayment status in August, 2005; . . .;X11 = the repayment status in April, 2005. The measurement scale for the repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above.<br>
X12-X17: Amount of bill statement (NT dollar). X12 = amount of bill statement in September, 2005; X13 = amount of bill statement in August, 2005; . . .; X17 = amount of bill statement in April, 2005.<br>
X18-X23: Amount of previous payment (NT dollar). X18 = amount paid in September, 2005; X19 = amount paid in August, 2005; . . .;X23 = amount paid in April, 2005.<br>

#### Important Note: -2 in PAY0~6 means he does not have any payment balance. Just like it is 0.

# 2.EDA

In [6]:
df.isnull().sum()

limit_bal                     0
sex                           0
education                     0
marriage                      0
age                           0
pay_0                         0
pay_2                         0
pay_3                         0
pay_4                         0
pay_5                         0
pay_6                         0
bill_amt1                     0
bill_amt2                     0
bill_amt3                     0
bill_amt4                     0
bill_amt5                     0
bill_amt6                     0
pay_amt1                      0
pay_amt2                      0
pay_amt3                      0
pay_amt4                      0
pay_amt5                      0
pay_amt6                      0
default_payment_next_month    0
dtype: int64

In [7]:
df.default_payment_next_month.value_counts(normalize=True)

0    0.7788
1    0.2212
Name: default_payment_next_month, dtype: float64

# 3.Splitting The Data

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)
len(df_train), len(df_val), len(df_test)

(18000, 6000, 6000)

In [10]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [11]:
y_train = df_train.default_payment_next_month.values
y_val = df_val.default_payment_next_month.values
y_test = df_test.default_payment_next_month.values

del df_train['default_payment_next_month']
del df_val['default_payment_next_month']
del df_test['default_payment_next_month']

# 4.Feature Importance: Mutual Information & Correlation

#### In our case there is no caterogical variables (strings).<br>So we should only use Correlation,so it's just for me

In [12]:
from sklearn.metrics import mutual_info_score

In [13]:
def mutual_info_payment_score(series):
    return mutual_info_score(series, df_full_train.default_payment_next_month)

In [14]:
categorical=['sex', 'education', 'marriage', 'pay_0', 'pay_2','pay_3', 'pay_4', 'pay_5', 'pay_6']
numerical=['limit_bal','age', 'bill_amt1', 'bill_amt2', 'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6', 'pay_amt1',
       'pay_amt2', 'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6']

In [15]:
mi = df_full_train[categorical].apply(mutual_info_payment_score)
mi.sort_values(ascending=False)

pay_0        0.076832
pay_2        0.050237
pay_3        0.036600
pay_4        0.032810
pay_5        0.030289
pay_6        0.025822
education    0.002990
marriage     0.000601
sex          0.000598
dtype: float64

In [16]:
df_full_train[numerical+categorical].corrwith(df_full_train.default_payment_next_month).abs().sort_values(ascending=False)

pay_0        0.324296
pay_2        0.266372
pay_3        0.232016
pay_4        0.215763
pay_5        0.203252
pay_6        0.184769
limit_bal    0.149247
pay_amt1     0.071736
pay_amt2     0.058978
pay_amt5     0.055625
pay_amt4     0.055338
pay_amt3     0.054129
pay_amt6     0.050185
sex          0.034700
education    0.027585
marriage     0.025817
bill_amt1    0.019395
age          0.016778
bill_amt2    0.014530
bill_amt3    0.013861
bill_amt4    0.008645
bill_amt5    0.005683
bill_amt6    0.005330
dtype: float64

# 5.One-Hot Encoding

#### Since we don't have Caterogical variables we can use (df.values) approach

In [17]:
from sklearn.feature_extraction import DictVectorizer

In [18]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

test_dict=df_test[categorical + numerical].to_dict(orient='records')
X_test=dv.transform(test_dict)

# 6.Training Logistic Regression With Scikit-Learn

In [19]:
from sklearn.linear_model import LogisticRegression

In [20]:
model = LogisticRegression(solver='lbfgs')
model.fit(X_train, y_train)

LogisticRegression()

In [21]:
y_pred = model.predict_proba(X_val)[:, 1]

In [22]:
payment_decision = (y_pred >= 0.5)
(y_val == payment_decision).mean()

0.7798333333333334

In [23]:
X=df_train.values
X

array([[ 80000,      1,      2, ...,   3000,  19707,  50000],
       [170000,      1,      1, ...,   3500,   7000,   5000],
       [ 50000,      1,      4, ...,    600,   1000,    404],
       ...,
       [400000,      2,      2, ...,    990,    964,    975],
       [110000,      1,      1, ...,   2000,   2000,   2000],
       [ 90000,      1,      2, ...,    800,   1000,    800]], dtype=int64)

In [24]:
model_1 = LogisticRegression(solver='lbfgs')
model_1.fit(X, y_train)

LogisticRegression()

In [25]:
X_val_1=df_val.values

In [26]:
y_pred_1 = model_1.predict_proba(X_val_1)[:, 1]

In [27]:
payment_decision_1 = (y_pred_1 >= 0.5)
(y_val == payment_decision_1).mean()

0.7798333333333334

The accuracy of our model is 0.78